In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Загружаем наборы данных

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
ratings.head(5)

In [ ]:
movies.head(5)

### Создаём матрицу

In [ ]:
def ratings_matrix(ratings):
    # создайте csr матрицу

R = ratings_matrix(ratings)

In [ ]:
R

In [ ]:
R.indices

In [ ]:
R.data

# Пример моделей коллаборативной фильтрации (CF)

Можно выделить основные варианты применения:

1. Поиск сходства между предметами или пользователями
2. Предикт рейтинга
3. Формирование Top-N


# 1. User-based

### 1. Определим из всей выборки пользователе, которых будем сравнивать с активным

In [ ]:
def create_model(rating_matrix, metric):
    """
    - создание модели с базовыми параметрами
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')

    model.fit(rating_matrix)
    return model

In [ ]:
def nearest_neighbors(rating_matrix, model):
    """
    :param rating_matrix : матрица рейтингов (nb_users, nb_items)
    :param model : модель knn
    """
    similarities, neighbors = model.kneighbors(rating_matrix)
    return similarities[:, 1:], neighbors[:, 1:]

In [ ]:
# метрику схожести используем Косинусную
model = create_model(rating_matrix=R, metric='cosine')
similarities, neighbors = nearest_neighbors(R, model)

### 2. Поиск элементов пользователя

In [ ]:
def find_candidate_items(userid):
    """
    Поиск элементов для переданного пользователя

    :param userid : пользователь id
    :param neighbors : схожесть между пользователями
    :return candidates : топ 10 элементов для пользователя
    """
    user_neighbors = neighbors[userid]

    activities = ratings.loc[ratings['userId'].isin(user_neighbors)]


    # сортируем элементы по частоте
    frequency = activities.groupby('movieId')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency['movieId']
    active_items = ratings.loc[ratings['userId'] == userid]['movieId'].to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:10]

    return candidates

### 3. Предикт рейтинга

In [ ]:
# средний рейтинг по всем
mean = ratings.groupby(by='userId', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userId')

# нормализация рейтинга
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

mean = mean.to_numpy()[:, 1]

In [ ]:
np_ratings = mean_ratings.to_numpy()

```predict``` рейтинга между пользователями по функции:

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in G_u}(r_{v,i}-\bar{r}_v)\cdot w_{u,v}}{\sum_{v\in G_u}|w_{u,v}|}.
\end{equation}

In [ ]:
def predict(userid, itemid):
    """
    предикт для пользователя userid рейтинга на элемент itemid.

    :param
        - userid : пользователь для предикта
        - itemid : элемент для предикта

    :return
        - r_hat : предикт
    """
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]

    # средний рейтинг
    user_mean = mean[userid]

    # поиск пользователей, которые имеют рейтинг по элементу 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]

    # поиск похожих пользователей
    simus = iratings[np.isin(iratings[:, 0], user_neighbors)]

    # отбор похожих пользователей, которые имеют рейтинг по выбранному элементу
    normalized_ratings = simus[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in simus[:, 0].astype('int')]
    sims = user_similarities[indexes]

    num = # подготовить произведение normalized_ratings и sims  np.dot(normalized_ratings, sims)
    den = # сумма всех sims, не учитывая знак np.sum(np.abs(sims))

    if num == 0 or den == 0:
        return user_mean

    # реализуем формулу предикста

    return r_hat

In [ ]:
def user2userPredictions(userid, pred_path):
    """
    Сделаем предикт для каждого пользователя и сохраним в файл prediction.csv

    :param
        - userid : пользователя id
        - pred_path : куда сохраняем
    """

    try:
        # поиск пользователей
        candidates = find_candidate_items(userid)

        # цикл по всем выбраным пользователям для предикта
        for itemid in candidates:

            # предикт для пользователя, по элементам
            r_hat = predict(userid, itemid)

            # сохраним
            with open(pred_path, 'a+') as file:
                line = '{},{},{}\n'.format(userid, itemid, r_hat)
                file.write(line)
    except IndexError:
        pass

In [ ]:
import sys
import os

def user2userCF():
    """
    Предикт для всех пользователей, даже с 1 рейтингом
    """
    # список всех пользователей
    users = ratings['userId'].unique()

    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()

    saved_predictions = 'predictions.csv'
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)

    for count, userid in enumerate(users):
        # делаем предикт
        user2userPredictions(userid, saved_predictions)
        _progress(count)

In [ ]:
user2userCF()

### 4. Top-N рекомендаций

Функция ```user2userRecommendation()``` делает отбор необходимых рекомендаций для пользователя

In [ ]:
def user2userRecommendation(userid):
    """
    Делаем предикт для пользователя
    """

    saved_predictions = 'predictions.csv'

    predictions = pd.read_csv(saved_predictions, sep=',', names=['userId', 'movieId', 'predicted_rating'])
    predictions = predictions[predictions.userId==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)

    List = pd.merge(List, movies, on='movieId', how='inner')

    return List

In [ ]:
user2userRecommendation(4)